In [15]:
# import the necessary libraries
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import pickle

In [16]:
#Load all the trained model, scaler, label_encoding, one_hot_encoding
model=load_model('ANN_model.h5')


#Load the scaler and encdoer
with open('encoder_gender.pkl','rb') as file:
    encoder_gender=pickle.load(file)
    
with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)
    
with open('one_hot_encoding_geography.pkl','rb') as file:
    one_hot_encoding_geography=pickle.load(file)
    
    

In [17]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [18]:
# One-hot encode 'Geography'
geo_encoded = one_hot_encoding_geography.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoding_geography.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\Syed Ali Zar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [19]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [20]:
## Encode categorical variables
input_df['Gender']=encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [21]:
## concatination one hot encoded 
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [22]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.47912542,  0.91678285,  0.09830529, -0.71248217, -0.26873812,
         0.78043148,  0.64635082,  0.96914252, -0.84682956,  0.99468081,
        -0.58607517, -0.56451866]])

In [23]:
## PRedict churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 531ms/step


array([[0.05290623]], dtype=float32)

In [24]:
prediction_proba = prediction[0][0]

In [25]:
prediction_proba

0.052906226

In [26]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
